In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sqlite3
import re

from functools import partial
from itertools import product

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

from sklearn.neighbors.kde import KernelDensity
from scipy.stats import norm

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def is_a_in_x(A, X):
    for i in range(len(X) - len(A) + 1):
        if A == X[i:i+len(A)]: return True
    return False

In [4]:
con = sqlite3.connect('soccer.sqlite')
c = con.cursor()

In [46]:
c.execute('select * from team')
[x[0] for x in c.description]

['teamID',
 'ligaID',
 'sofifaTeamID',
 'patchID',
 'name',
 'geschwindigkeit',
 'dribbling',
 'passen_aufbau',
 'passen_chancen',
 'flanken',
 'schussverhalten',
 'druck',
 'aggressivitat',
 'verschieben']

In [29]:
con.commit()

In [30]:
jersey_sum = [x for x in c.execute("""
SELECT m.FTR, p1.jersey+ p2.jersey+ p3.jersey+ p4.jersey+ p5.jersey+p6.jersey+p7.jersey+p8.jersey+p9.jersey+p10.jersey+p11.jersey as homesum,
p19.jersey+p20.jersey+p21.jersey+p22.jersey+p23.jersey+p24.jersey+p25.jersey+p26.jersey+p27.jersey+p28.jersey+p29.jersey as awaysum
FROM match m 
join player p1 on m.homePlayer1 = p1.playerID
join player p2 on m.homePlayer2 = p2.playerID
join player p3 on m.homePlayer3 = p3.playerID
join player p4 on m.homePlayer4 = p4.playerID
join player p5 on m.homePlayer5 = p5.playerID
join player p6 on m.homePlayer6 = p6.playerID
join player p7 on m.homePlayer7 = p7.playerID
join player p8 on m.homePlayer8 = p8.playerID
join player p9 on m.homePlayer9 = p9.playerID
join player p10 on m.homePlayer10 = p10.playerID
join player p11 on m.homePlayer11 = p11.playerID
join player p12 on m.homePlayer12 = p12.playerID
join player p13 on m.homePlayer13 = p13.playerID
join player p14 on m.homePlayer14 = p14.playerID
join player p15 on m.homePlayer15 = p15.playerID
join player p16 on m.homePlayer16 = p16.playerID
join player p17 on m.homePlayer17 = p17.playerID
join player p18 on m.homePlayer18 = p18.playerID
join player p19 on m.awayPlayer1 = p19.playerID
join player p20 on m.awayPlayer2 = p20.playerID
join player p21 on m.awayPlayer3 = p21.playerID
join player p22 on m.awayPlayer4 = p22.playerID
join player p23 on m.awayPlayer5 = p23.playerID
join player p24 on m.awayPlayer6 = p24.playerID
join player p25 on m.awayPlayer7 = p25.playerID
join player p26 on m.awayPlayer8 = p26.playerID
join player p27 on m.awayPlayer9 = p27.playerID
join player p28 on m.awayPlayer10 = p28.playerID
join player p29 on m.awayPlayer11 = p29.playerID
join player p30 on m.awayPlayer12 = p30.playerID
join player p31 on m.awayPlayer13 = p31.playerID
join player p32 on m.awayPlayer14 = p32.playerID
join player p33 on m.awayPlayer15 = p33.playerID
join player p34 on m.awayPlayer16 = p34.playerID
join player p35 on m.awayPlayer17 = p35.playerID
join player p36 on m.awayPlayer18 = p36.playerID""")]

In [31]:
spot = {'H':0, 'D':1, 'A':2}
res = [[],[],[]]
for match in jersey_sum:
    res[spot[match[0]]].append(match[1]-match[2])

In [4]:
raw = [x for x in c.execute('SELECT * from match')]

In [3]:
x_data = []
y_data = []

for m in raw:
    home_stats = np.array([x[5:] for x in c.execute('SELECT * FROM team WHERE teamID IS ?', (m[3],) )][0])
    away_stats = np.array([x[5:] for x in c.execute('SELECT * FROM team WHERE teamID IS ?', (m[4],) )][0])

    m = np.array(m)
    for _ in range(1):
        players = []
        for pid in m[6:17]:
            if pid == -1 or pid == '-1':
                players += [-1]*37
            else:
                players += [a[4:-1] for a in c.execute('SELECT * FROM player WHERE playerID IS ?', (pid,))][0]

        for pid in m[24:35]:
            if pid == -1:
                players += [-1]*37
            else:
                pnew =  [a[4:-1] for a in c.execute('SELECT * FROM player WHERE playerID IS ?', (pid,))]
                if len(pnew) == 1:
                    players += pnew[0]
                else:
                    players += [-1]*37


        if np.random.randint(2):
            players = [np.random.normal(attr, 2) for attr in players]

        x_data.append([*home_stats,*np.array(players).flatten(),*away_stats])

        if m[42] > m[43]:
            y_data.append([1,0,0])
        elif m[42] < m[43]:
            y_data.append([0,0,1])
        else:
            y_data.append([0,1,0])

        #shuffle and again
        np.random.shuffle(m[7:17])
        np.random.shuffle(m[25:34])


# In[99]:

# for i,t in enumerate(x_data):
#     for j,l in enumerate(t):
#         if l == '':
#             x_data[i][j] = -1
#         elif re.search('^[0-9][0-9]*[+,-][0-9]*',str(l)):
#             k = l.split('-')[0].split('+')[0]
#             x_data[i][j] = k

In [112]:
corr = [[],[],[]]
for i,match in enumerate(x_data):
    if is_a_in_x([-1 for x in range(30)], match[9:-9]):
        continue
    fifa_sum = np.mean(match[9:416])-np.mean(match[416:-9])
    if y_data[i][0] == 1:
        corr[0].append(fifa_sum)
    elif y_data[i][1] == 1:
        corr[1].append(fifa_sum-)
    else:
        corr[2].append(fifa_sum)

In [32]:
corr = res

In [33]:
hist, edges = np.histogram(corr[0], density=True, bins=250)
hist1, edges1 = np.histogram(corr[1], density=True, bins=250)
hist2, edges2 = np.histogram(corr[2], density=True, bins=250)

In [34]:
x_val = np.linspace(min([min(x) for x in corr]), max([max(x) for x in corr]), 500)
mean, std = norm.fit(corr[0])
y_val = norm.pdf(x_val, mean, std)

In [35]:
p1 = figure(title='heimsieg')
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
p1.line(x_val, y_val,  line_color="#D95B43", line_width=2, alpha=0.7, legend="PDF")
show(p1)

In [36]:
mean1, std1 = norm.fit(corr[1])
y_val1 = norm.pdf(x_val, mean1, std1)
p2 = figure(title='draw')
p2.quad(top=hist1, bottom=0, left=edges1[:-1], right=edges1[1:],
        fill_color="#036564", line_color="#033649")
p2.line(x_val, y_val1,  line_color="#D95B43", line_width=2, alpha=0.7, legend="PDF")
show(p2)

In [37]:
mean2, std2 = norm.fit(corr[2])
y_val2 = norm.pdf(x_val, mean2, std2)
p3 = figure(title='ausw&auml;rtssieg')
p3.quad(top=hist1, bottom=0, left=edges1[:-1], right=edges1[1:],
        fill_color="#036564", line_color="#033649")
p3.line(x_val, y_val2,  line_color="#D95B43", line_width=2, alpha=0.7, legend="PDF")
show(p1)

In [38]:
p4 = figure(title='comparison')
p4.line(x_val, y_val, line_color='red', legend = 'Home')
p4.line(x_val, y_val1, line_color='green', legend = 'Draw')
p4.line(x_val,y_val2, line_color='blue', legend = 'Away')
show(p4)

In [39]:
print(mean2,std2)
print(mean, std)
print(mean1, std1)

-1.85157930488 54.6164693594
-3.46024858425 54.7384829875
-2.2288594833 54.661767333


In [40]:
[len(x) for x in corr]

[13597, 7935, 8833]

In [43]:
sum([len(x) for x in corr])

30365

In [25]:
design_matrix = []
for match in raw:     

SyntaxError: unexpected EOF while parsing (<ipython-input-25-a6f93ebd6d71>, line 2)

In [5]:
team_dict = {k:v for k,v in zip([x[0] for x in c.execute('select sofifaTeamID from team where ligaID IS 19 group by sofifaTeamID')], range(len([x[0] for x in c.execute('select sofifaTeamID from team where ligaID IS 19 group by sofifaTeamID')])))}

In [6]:
team_list = [x[0] for x in c.execute('select teamID from team where ligaID is 19')]

In [14]:
reg_x = []
reg_y = []
teamNr = len(team_dict.keys())
for match in raw:
    if match[3] in team_list:
        line = np.zeros(teamNr+1)
        line[team_dict[int(match[3]/1000000)]] = 1
        line[team_dict[int(match[4]/1000000)]] = -1
        line[-1] = 1

        reg_x.append(line)
        reg_x.append(line)

        if match[44] == 'A':
             reg_y.append(1)
             reg_y.append(1)

        elif match[44] == 'D':
             reg_y.append(1)
             reg_y.append(0)     
        else:
             reg_y.append(0)
             reg_y.append(0)

In [8]:
from sklearn.linear_model import LogisticRegression

In [15]:
est= LogisticRegression(fit_intercept=False)

In [16]:
est.fit(reg_x, reg_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
est.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': False,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [60]:

skill = np.exp(-1*est.coef_)[0]
skill = skill*np.linalg.norm(skill)

In [61]:
skill

array([ 40.08338948,  22.3816073 ,  10.9602614 ,   7.4561041 ,
         7.40779564,   5.87627277,   7.89797036,  15.7472095 ,
        12.36887834,   7.33581229,   8.02950428,   5.94419856,
         3.21721779,   6.77730766,   9.22806072,   6.72570805,
         9.87481441,   7.19117245,   7.76068204,   8.43420632,
         5.35300859,   7.80738306,   4.32427895,   3.80311128,
         5.21740042,   4.7463867 ,   6.46388884,  21.54651526,  11.33733889])

In [62]:
names = {x[1]:x[0] for x in c.execute('select name, sofifaTeamID from team where ligaID is 19 group by sofifaTeamID')}

In [63]:
result = np.array([(id_, names[id_] , skill[team_dict[id_]]) for id_ in team_dict.keys()], dtype = [('id',int), ('name','S20'), ('skill',float)])

In [64]:
np.sort(result, order=['skill', 'id'] )[::-1]

array([(    21, b'FC Bayern Munich',  40.08338948),
       (    22, b'Borussia Dortmund',  22.3816073 ),
       (112172, b'RB Leipzig',  21.54651526),
       (    32, b'Bayer 04 Leverkusen',  15.7472095 ),
       (    34, b'FC Schalke 04',  12.36887834),
       (    23, b'Borussia Monchenglad',  10.9602614 ),
       (   175, b'VfL Wolfsburg',   9.87481441),
       (   169, b'1. FSV Mainz 05',   9.22806072),
       ( 10029, b'TSG 1899 Hoffenheim',   8.43420632),
       (    38, b'SV Werder Bremen',   8.02950428),
       (    31, b'1. FC Koln',   7.89797036),
       (100409, b'FC Augsburg',   7.80738306),
       (  1824, b'Eintracht Frankfurt',   7.76068204),
       (    25, b'SC Freiburg',   7.4561041 ),
       (    28, b'Hamburger SV',   7.40779564),
       (    36, b'VfB Stuttgart',   7.33581229),
       (   485, b'Hannover 96',   7.19117245),
       (   166, b'Hertha BSC Berlin',   6.77730766),
       (   171, b'1. FC Nurnberg',   6.72570805),
       (111239, b'FC Ingolstadt 04',   6

In [71]:
def pred_prob(id1, id2):
    line = np.zeros(teamNr+1)
    line[team_dict[id1]] = 1
    line[team_dict[id2]] = -1
    line[-1] = 1
    return est.predict_proba([line])

In [74]:
pred_prob(165,21)

array([[ 0.10121129,  0.89878871]])